# MoCap Arena

---

In [1]:
# Importing modules...
import numpy as np
import scipy as sp
from scipy.interpolate import CubicSpline
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist
import cv2
import socket

import sys
sys.path.append('../..') # Go back to base directory

from modules.vision.camera import Camera
from modules.vision.multiple_view import MultipleView

from modules.plot.graph import Graph

# Instanciating Multiple View Object

To generate the data regarding the mathematical model of multiple vision triangulation in *CoppeliaSim*, it will be instanciated a Multiple View Object, composed of a set of Camera Obejcts that matches the Vision Sensors' parameters. 

---

In [2]:
n_clients = 4
cameras = [] # Camera objects list

# Object matrix of Camera 0
base_matrix = np.array([[-7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 1.46327395e-13, -7.07106781e-01, -7.07106781e-01, 2.50000000e+00]])

for ID in range(n_clients):
    # Spread all cameras uniformely in a circle around the arena
    R = np.array(sp.spatial.transform.Rotation.from_euler('z', (360 / n_clients) * ID, degrees=True).as_matrix())
    object_matrix = R @ base_matrix

    cameras.append(Camera(# Intrinsic Parameters
                          resolution=(720,720), 
                          fov_degrees=60.0,     
          
                          # Extrinsic Parameters
                          object_matrix=object_matrix
          
                          # Rational Lens Distortion Model
                          # distortion_model='rational',
                          # distortion_coefficients=np.array([0.014, -0.003, -0.0002, -0.000003, 0.0009, 0.05, -0.007, 0.0017]), 
                          
                          # Fisheye Lens Distortion Model
                          # distortion_model='fisheye',
                          # distortion_coefficients=np.array([0.395, 0.633, -2.417, 2.110]),
          
                          # Image Noise Model
                          # snr_dB=13
                          ))
    
multiple_view = MultipleView(cameras)

# Creating the Server's UDP Socket

---

In [3]:
print(f'[SERVER] Creating socket...')

# Try to create server socket
try: 
    server_socket = socket.socket(socket.AF_INET,    # Internet
                                  socket.SOCK_DGRAM) # UDP
    print(f'[SERVER] Socket successfully created')
    
except socket.error as err: 
    print(f'[SERVER] Socket creation failed with error {err}\n')
    print(f'[SERVER] Quitting code...')
    exit()

server_ip = '127.0.0.1' # Server IP
server_port = 8888      # Server Port
server_address = (server_ip, server_port) 

server_socket.bind(server_address)
print(f'[SERVER] Bound to port {server_port}')

buffer_size = 1024 # Size of the messages in bytes

[SERVER] Creating socket...
[SERVER] Socket successfully created
[SERVER] Bound to port 8888


# Declaring the Controller's UDP Socket

---

In [4]:
# Controller socket info
controller_ip = '127.0.0.1' # Controller IP
controller_port = 7777      # Controller Port
controller_address = (controller_ip, controller_port)

# Sending Capture Info

---

In [5]:
simulation_time = 10 # In seconds

# Send capture info to the controller
message = f'{n_clients}, {simulation_time}'

message_bytes = message.encode()

server_socket.sendto(message_bytes, controller_address)
print(f'[SERVER] Capture info sent')

# Wait for controller to setup the info
server_socket.recvfrom(buffer_size)
print(f'[SERVER] Clients created')

[SERVER] Capture info sent
[SERVER] Clients created


# Sending Vision Sensor Parameters

---

In [6]:
def coppelia_vision_sensor_wrapper(camera):
    parameter_array = np.array([# Options
                                2+4, # Bit 1 set: Perspective Mode
                                     # Bit 2 set: Invisible Viewing Frustum 
                                                                     
                                # Integer parameters
                                camera.resolution[0], 
                                camera.resolution[1],
                                0, # Reserved
                                0, # Reserved

                                # Float parameters
                                0.01, # Near clipping plane in meters
                                10, # Far clipping plane in meters
                                camera.fov_radians, # FOV view angle in radians
                                0.1, # Sensor X size
                                0.0, # Reserved
                                0.0, # Reserved
                                0.0, # Null pixel red-value
                                0.0, # Null pixel green-value
                                0.0, # Null pixel blue-value
                                0.0, # Reserved
                                0.0, # Reserved
                                ], 
                               dtype=np.float64)

    transformation_array = camera.coppeliasim_object_matrix.ravel()
    
    buffer = np.concatenate((parameter_array, transformation_array)).tobytes()

    return buffer

print(f'[SERVER] Sending Vision Sensors\' parameters')

for ID, C in enumerate(multiple_view.cameras):
    # Wrap vision sensor parameters
    buffer = coppelia_vision_sensor_wrapper(C)

    server_socket.sendto(buffer, controller_address)

    # Wait for controller to setup the info
    server_socket.recvfrom(buffer_size)
    print(f'\tVision Sensor {ID} created')

[SERVER] Sending Vision Sensors' parameters
	Vision Sensor 0 created
	Vision Sensor 1 created
	Vision Sensor 2 created
	Vision Sensor 3 created


# Sending trigger

---

In [7]:
# Wait for controller to setup the scene
server_socket.recvfrom(buffer_size)
print(f'[SERVER] Scene ready')

# Send trigger
server_socket.sendto(''.encode(), controller_address)
print(f'[SERVER] Trigger sent!')

[SERVER] Scene ready
[SERVER] Trigger sent!


# Handshaking Clients

---

In [8]:
client_addresses = {}

print(f'[SERVER] Waiting for clients...')

# Address lookup 
while len(client_addresses.keys()) < n_clients: # Until all clients are identified
    message_bytes, address = server_socket.recvfrom(buffer_size)

    try:
        ID = int(message_bytes.decode()) # Decode message

    except: # Invalid message for decoding
        continue # Look for another message

    client_addresses[address] = ID

    print(f'\tClient {ID} connected')

print(f'[SERVER] All clients connected!')

[SERVER] Waiting for clients...
	Client 0 connected
	Client 1 connected
	Client 2 connected
	Client 3 connected
[SERVER] All clients connected!


# Capture Data Class

---

In [9]:
def order_by_proximity(previous_blobs, current_blobs):
    ordered_current_blobs = np.empty_like(current_blobs)

    for current_blob in current_blobs:
        distances = [] # Distances from a previous blob to all current blobs

        for previous_blob in previous_blobs:
            distances.append(np.linalg.norm(previous_blob - current_blob))

        new_index = np.argmin(np.array(distances)) # The match will be made for the shortest point to point distance

        ordered_current_blobs[new_index] = current_blob

    return ordered_current_blobs

# Data structure for blob interpolation
class CaptureData:
    def __init__(self, 
                 n_blobs=1, # Number of expected blobs for interpolation
                 window=10, # The minimum ammount of data points for interpolating 
                 step=0.01, # Time step for interpolation in seconds
                 capture_time=10, # Capture time in seconds
                 ):
        
        # Interpolation Parameters
        self.n_blobs = n_blobs
        self.interpolation_window = window
        self.step = step
        self.capture_time = capture_time
        self.last_interpolation = 0

        # Raw data - how it comes from the clients
        self.raw_blobs = []
        self.raw_PTS = []

        # Interpolated data - how it should be triangulated
        self.int_PTS = np.arange(0.0, self.capture_time, self.step)
        self.int_blobs = np.full((self.int_PTS.size, n_blobs, 2), -1.0) 
       
    def add_data(self, blobs, PTS):
        # Do not add data if PTS is out of recording range
        if PTS > self.capture_time:
            return 

        # Check if it's not empty
        if self.raw_PTS:
            # Do not add data if PTS is already added, avoid repeated messaging
            if PTS == self.raw_PTS[-1]:
                return 

        self.raw_PTS.append(PTS)

        # Ordering blobs by proximity for correct interpolation 
        if self.raw_blobs:  
            blobs = order_by_proximity(self.raw_blobs[-1], blobs)

        self.raw_blobs.append(blobs)

        # Enough points to interpolate?
        if len(self.raw_PTS) >= self.interpolation_window:
            # Get the window last blob coordinate data
            raw_PTS = np.array(self.raw_PTS[-self.interpolation_window:])
            raw_blobs = np.array(self.raw_blobs[-self.interpolation_window:])

            start = self.last_interpolation     # Start index of interpolated PTS  
            end = int(raw_PTS[-1] // self.step) # Final index of interpolated PTS  

            for blob in range(self.n_blobs):
                # Extracting blob coordinate history in the interpolation window
                # The slicing works like: raw_blob = raw_blobs[PTS, blob, axis]
                raw_blob = raw_blobs[:, blob, :]

                # Generating a cubic spline that represents blob trajectory 
                blob_trajectory = CubicSpline(raw_PTS, raw_blob)
                
                # Get blob tracjectory in the interpolated timestamps that are not yet interpolated
                interpolated_blob = blob_trajectory(self.int_PTS[start:end+1]) # End of slice is exclusive!

                # Add interpolated blobs to data structure
                self.int_blobs[start:end+1, blob, :] = interpolated_blob

            # Updating last interpolation to the next PTS 
            self.last_interpolation = end + 1

n_blobs = 1 # Number of expected markers
window = 3 # The minimum ammount of points for interpolating 
throughput = 20 # Triangulated scenes per second
step = 1 / throughput 
capture_time = 10 # Total capture time

capture_data = [CaptureData(n_blobs, window, step, capture_time) for _ in range(n_clients)]

last_triangulation = 0
triangulated_markers = np.empty((capture_data[0].int_PTS.size, n_blobs, 3)) 

# Running Simulation

---

In [10]:
verbose = False

timeout = 5 # In seconds
server_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Breaks in the timeout
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server_socket.recvfrom(buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]}):')

    # Decode message
    try:
        message = np.frombuffer(message_bytes, dtype=np.float64)

    except:
        if verbose: print('> Couldn\'t decode message')

        continue # Jump to wait for the next message

    # Empty message
    if not message.size:
        if verbose: print(f'\tEmpty message')

        continue # Jump to wait for the next message

    # Parsing message
    PTS = message[-1] # Last element of the message contains PTS

    # Valid messages only comes in 2 coordinates (u and v) per blob and the PTS
    if message.size !=  2 * n_blobs + 1:

        if message.size == 1: # Only PTS
            if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

        else: 
            if verbose: print(f'\tWrong blob count or corrupted message - {PTS :.3f} s')

        continue # Jump to wait for the next message

    # Extracting 
    detected_blobs = message[:-1].reshape(2, -1).T # All but last element are the blob coordinates

    # Print blobs
    if verbose:
        print(f'\tDetected Blobs - {PTS :.3f} s')
        print('\t' + str(detected_blobs).replace('\n', '\n\t'))

    # Save data
    capture_data[ID].add_data(detected_blobs, PTS)

    # Check for available interpolated data
    available = []

    for available_ID, CD in enumerate(capture_data):
        # Is there interpolated data? Non interpolated blobs are negative!
        if np.any(CD.int_blobs[last_triangulation] >= 0): 
            available.append(available_ID)

    # If at least two cameras have interpolated data, triangulate 
    if len(available) >= 2:

        # Get the first two available
        pair = available[:2]

        distorted_centroids_pair = [capture_data[pair[0]].int_blobs[last_triangulation].T,
                                    capture_data[pair[1]].int_blobs[last_triangulation].T]

        buffer = multiple_view.triangulate_by_pair(pair, distorted_centroids_pair).astype(np.float64).ravel().tobytes()
        
        server_socket.sendto(buffer, controller_address)

        # Update last triangulation flag
        if last_triangulation < triangulated_markers.shape[0] - 1:
            last_triangulation += 1 


[SERVER] Timeout set to 5 seconds


[SERVER] Timed Out!


In [11]:
# Create graph
messages_plot = Graph(title='Valid Received Messages',
                      axis_title=('PTS', 'Clients'))

# Adding received client messages
for ID in range(n_clients):
    client_messages = np.array(capture_data[ID].raw_PTS)
    messages_per_PTS = np.vstack((client_messages, np.full(client_messages.shape, ID)))
    messages_plot.add_points(messages_per_PTS, f'Client {ID}')

# Plot image
messages_plot.figure.show(renderer='notebook_connected')

# Playing Back the Camera's Feed

The following cell will replicate a real time camera feed of the simulation. Change the `ID` parameter to switch between camera views.

---

In [13]:
# Playback a camera image feed in real time
ID = 0 # Camera ID to be watched

for (PTS_data, blob_data) in [(capture_data[ID].raw_PTS, capture_data[ID].raw_blobs), 
                              (capture_data[ID].int_PTS, capture_data[ID].int_blobs)]:
    
    # Converting to np arrays
    PTS_data = np.array(PTS_data)
    blob_data = np.array(blob_data)
    
    # Generating frames
    images = []
    for PTS, blobs in zip(PTS_data, blob_data):
        image = np.zeros(cameras[ID].resolution)

        for blob in blobs:
            image = cv2.circle(image, blob.astype(int), 2, 1, -1) 
            
        images.append(image)

    # Getting delay between each frame
    delays = PTS_data[1:] - PTS_data[:-1]

    # Playing the animation
    for delay, image in zip(delays, images):
        cv2.imshow(f'Camera {ID}', image)
        cv2.waitKey(int(1e3 * delay))

    # Closing all open windows 
    cv2.destroyAllWindows()
